In [2]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
import boto3

bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

In [11]:
def insights_prompt_withkb_generator(insurance_provider, pharmacy_name, insurance_type="Commercial", initiation_source="Hub", territory="east"):
    prompt_claude3_sonnet = f'''Human:
    
You are an agent assistant who is recommending a patient the best Specialty Pharmacy.
The best Specialty pharmacy will have the least number of days in time to fill for each while ignoring the conversion percentage.
Compare the pharmacy data available for specified payer and Insurance type.

The patient's current Time to Fill is the value of Time to Fill for 
Payer {insurance_provider}, Insurance type {insurance_type}, 
Specialty Pharmacy {pharmacy_name}, Initiation Source "{initiation_source}" and {territory} east.

Determine the patient's new options by finding the Specialty Pharmacies with the lowest value of Time to Fill for 
Payer {insurance_provider}, Insurance type {insurance_type}, Initiation Source "{initiation_source}" and {territory} east.

Based on the above, determine if the patient's new options would provide a lower time to fill than the current time to fill.

If so, recommend the new Specialty Pharmacy to the patient.
If not, advise the patient to stay with their current Specialty pharmacy.


Assistant:
'''
    return prompt_claude3_sonnet


response_withkb = bedrock_agent_runtime_client.retrieve_and_generate(
  input={
    'text': insights_prompt_withkb_generator("Express Scripts", "BioPlus", "Commercial", "Hub", "East")
  },
  retrieveAndGenerateConfiguration={
    'knowledgeBaseConfiguration': {
        'knowledgeBaseId': 'HZ2VTAKFFD',
        'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
    },
    'type': 'KNOWLEDGE_BASE'
  }
)

print("[ Generated Insights ]")
print(response_withkb["output"]["text"])

[ Generated Insights ]
The patient's current Time to Fill is 10 days for Payer Express Scripts, Insurance type Commercial, Specialty Pharmacy BioPlus, Initiation Source "Hub" in the East territory (source 5).

The Specialty Pharmacies with the lowest Time to Fill for Payer Express Scripts, Insurance type Commercial, Initiation Source "Hub" in the East territory are:
- Accredo: 6 days (source 5)
- Elixir: 7 days (source 5)
- Senderra Rx: 9 days (source 5) Since Accredo and Elixir have lower Time to Fill values (6 and 7 days respectively) compared to the patient's current 10 days, I would recommend the patient switch to either Accredo or Elixir as their new Specialty Pharmacy to reduce their Time to Fill.
